In [3]:
import os
import numpy as np 
import pandas as pd 
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from shutil import copyfile
from os import getcwd
print(tf.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.12.0


In [4]:
TRAINING_DIR ="/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres"

train_dataset = image_dataset_from_directory(TRAINING_DIR, validation_split=0.2,label_mode='categorical', subset="training", seed=42, image_size=(256, 256),
                                  
                                             batch_size=16)

val_dataset = image_dataset_from_directory(TRAINING_DIR, validation_split=0.2,label_mode='categorical', subset="validation", seed=42, image_size=(256, 256),
                                           batch_size=16)

# train_dataset = train_dataset.map(lambda x, y: (datagen.flow(x, batch_size=64), y))
# train_dataset

Found 1856 files belonging to 2 classes.
Using 1485 files for training.
Found 1856 files belonging to 2 classes.
Using 371 files for validation.


In [5]:

#################TRY DIFFERENT TRANSFER LEARNING MODELS########################
model_name='resnet101'
TL_layer = tf.keras.applications.resnet.ResNet101(input_shape=(256, 256, 3), 
                                                              include_top=False, weights='imagenet')
TL_layer.trainable = False
#################################################################################
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(256, 256, 3)),
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomHeight(0.1),
    tf.keras.layers.experimental.preprocessing.RandomWidth(0.1),
    tf.keras.layers.experimental.preprocessing.Resizing(256,256),
    TL_layer,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

old =0
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                #change model name here
                model.save(model_name+'.h5',overwrite=True)
                old = logs.get('val_acc')
                print("\n new best found threshold now = ",old)
                
saver =myCallback()
history = model.fit_generator(train_dataset,
                              epochs=200,
                              verbose=1,
                              validation_data=val_dataset,callbacks=[saver])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3491532401.py, line 15)

In [ ]:
# CUSTOM MODEL IMPLEMENTATION
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(256, 256, 3)),
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomHeight(0.1),
    tf.keras.layers.experimental.preprocessing.RandomWidth(0.1),
    tf.keras.layers.experimental.preprocessing.Resizing(256,256),
    tf.keras.layers.Conv2D(1024, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Conv2D(512, (3, 3), padding = 'same', activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3),  activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3),   activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3),  activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

old =0
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                #change model name here
                model.save('model_tire_RESNET101.h5',overwrite=True)
                old = logs.get('val_acc')
                print("\n new best found threshold now = ",old)
                
saver =myCallback()
history = model.fit_generator(train_dataset,
                              epochs=200,
                              verbose=1,
                              validation_data=val_dataset,callbacks=[saver])

In [ ]:
history = model.fit_generator(train_dataset,
                              epochs=200,
                              verbose=1,
                              validation_data=val_dataset,callbacks=[saver])

In [ ]:

# read the image
# img = cv2.imread('./data/tire-dataset/flat.class/00150.jpg')
img = cv2.imread('./data/source/defective/Defective (7).jpg')
class_index = 1
num_classes = 2

# format it to be in the RGB colorspace
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

# resize to 300x300 and normalize pixel values to be in the range [0, 1]
img = cv2.resize(img, (256, 256)) / 255.0

# add a batch dimension in front
image = np.expand_dims(img, axis=0)
plt.figure(figsize=(8, 8))
plt.imshow(img)
plt.axis('off')
plt.show()
# Siberian Husky's class ID in ImageNet



# convert to one hot representation to match our softmax activation in the model definition
expected_output = tf.one_hot([class_index] * image.shape[0], num_classes)

with tf.GradientTape() as tape:
    # cast image to float
    inputs = tf.cast(image, tf.float32)

    # watch the input pixels
    tape.watch(inputs)

    # generate the predictions
    predictions = model(inputs)

    # get the loss
    loss = tf.keras.losses.categorical_crossentropy(
        expected_output, predictions
    )

# get the gradient with respect to the inputs
gradients = tape.gradient(loss, inputs)
grayscale_tensor = tf.reduce_sum(tf.abs(gradients), axis=-1)

normalized_tensor = tf.cast(
    255
    * (grayscale_tensor - tf.reduce_min(grayscale_tensor))
    / (tf.reduce_max(grayscale_tensor) - tf.reduce_min(grayscale_tensor)),
    tf.uint8,
)

# remove the channel dimension to make the tensor a 2d tensor
normalized_tensor = tf.squeeze(normalized_tensor)

gradient_color = cv2.applyColorMap(normalized_tensor.numpy(), cv2.COLORMAP_HOT)

gradient_color = gradient_color / 255.0
super_imposed = cv2.addWeighted(img, 0.8, gradient_color, 0.8, 0.0)

plt.figure(figsize=(8, 8))
plt.imshow(super_imposed)
plt.axis('off')
plt.show()
plt.figure(figsize=(8, 8))
plt.imshow(normalized_tensor)
plt.axis('off')
plt.show()
# print(expected_output,np.argmax(tf.keras.activations.softmax(predictions)))

# # Set a threshold for high activations
# threshold = 0.5  # Adjust this threshold as needed

# # Filter out pixels below the threshold
# high_activation_pixels = normalized_tensor.numpy() > threshold

# # Create a mask for high activations
# high_activation_mask = np.zeros_like(img)
# high_activation_mask[high_activation_pixels] = img[high_activation_pixels]

# plt.figure(figsize=(8, 8))
# plt.imshow(high_activation_mask)
# plt.axis('off')
# plt.show()